In [2]:
import sqlite3
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host='localhost', port=1521, sid='xe')
conn = cx_Oracle.connect('dlow123', 'tmvjswl', dsn=oracle_dsn)

# SQLITE

In [13]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def __str__(self):
        return 'name:{}\nphone:{}\nemail:{}\nage:{}\ngrade:{}\netc:{}\n'.format(self.name, self.phone, self.email, self.age, self.grade, self.etc)
    def as_dict(self):
        return {'name':self.name, 'phone':self.phone, 'email':self.email, 'age':self.age, 'grade':self.grade, 'etc':self.etc}
def to_member(*row):
    return Member(row[0],row[1],row[2],row[3],row[4],row[5])

In [14]:
def insert_member():
    cursor = conn.cursor()
    name = input('등록하실분의 이름을 입력하세요 > ')
    phone = input('등록하실분의 전화번호을 입력하세요 > ')
    email = input('등록하실분의 이메일을 입력하세요 > ')
    while True:
        try:
            age = int(input('등록하실분의 나이을 정수로 입력하세요 > '))
            grade = int(input('등록하실분의 등급을 정수로 입력하세요 > '))
            break
        except:
            print('정수로 다시 입력해주세요')
    etc = input('등록하실분의 기타사항을 입력하세요 > ')
    cursor.execute('INSERT INTO MEMBER VALUES (:name, :phone, :email, :age, :grade, :etc)', Member(name, phone, email, age, grade, etc).as_dict())
    conn.commit()
    print('입력 성공!')
    cursor.close()

In [15]:
def select_all():
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    for member in members:
        print(to_member(*member))
    cursor.close()

In [16]:
def select_name():
    cursor = conn.cursor()
    name = input('검색하실분의 이름을 작성해 주세요')
    cursor.execute('SELECT * FROM MEMBER WHERE NAME = UPPER(:name)',{'name':name})
    members = cursor.fetchall()
    for member in members:
        print(to_member(*member))
    cursor.close()

In [17]:
def delete_email():
    cursor = conn.cursor()
    email = input('삭제하실분의 이메일을 작성해 주세요')
    cursor.execute('DELETE FROM MEMBER WHERE EMAIL = :email',{'email':email})
    conn.commit()
    if cursor.rowcount >= 1:
        print('삭제완료되었습니다')
    cursor.close()

In [18]:
def save_csv():
    import csv
    cursor = conn.cursor()
    save_member_list = []
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    for member in members:
        save_member_list.append(to_member(*member).as_dict())
    fieldnames = list(save_member_list[0].keys())
    filename = input('저장할 CSV 파일 이름은 ?')
    try :
        with open('data/{}.csv'.format(filename), 'w', newline='', encoding='UTF-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
            dict_writer.writeheader()
            dict_writer.writerows(save_member_list)
    except Exception as e:
        print('데이터 내보내기 예외 :', e)
    cursor.close()

In [19]:
def close_sql():
    conn.close()
    print('프로그램이 종료되었습니다')

In [20]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break

In [21]:
if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data/ch10_data.db')
    main()

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 2
name:홍길동
phone:010-9999-9999
email:h@h.com
age:20
grade:3
etc:바보

name:김길동
phone:010-9999-9999
email:k@k.com
age:25
grade:2
etc:멍총이

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 3
검색하실분의 이름을 작성해 주세요홍길동
name:홍길동
phone:010-9999-9999
email:h@h.com
age:20
grade:3
etc:바보

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 1
등록하실분의 이름을 입력하세요 > 아무개
등록하실분의 전화번호을 입력하세요 > 010-2222-2222
등록하실분의 이메일을 입력하세요 > 2@2.com
등록하실분의 나이을 정수로 입력하세요 > 33
등록하실분의 등급을 정수로 입력하세요 > 0
등록하실분의 기타사항을 입력하세요 > 몰랑
입력 성공!
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 2
name:홍길동
phone:010-9999-9999
email:h@h.com
age:20
grade:3
etc:바보

name:김길동
phone:010-9999-9999
email:k@k.com
age:25
grade:2
etc:멍총이

name:아무개
phone:010-2222-2222
email:2@2.com
age:33
grade:0
etc:몰랑

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 4
삭제하실분의 이메일을 작성해 주세요2@2.com
삭제완료되었습니다
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료


# ORACLE

In [4]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def __str__(self):
        return 'name:{}\nphone:{}\nemail:{}\nage:{}\ngrade:{}\netc:{}\n'.format(self.name, self.phone, self.email, self.age, self.grade, self.etc)
    def as_dict(self):
        return {'name':self.name, 'phone':self.phone, 'email':self.email, 'age':self.age, 'grade':self.grade, 'etc':self.etc}
def to_member(*row):
    return Member(row[0],row[1],row[2],row[3],row[4],row[5])

In [5]:
def insert_member():
    cursor = conn.cursor()
    name = input('등록하실분의 이름을 입력하세요 > ')
    phone = input('등록하실분의 전화번호을 입력하세요 > ')
    email = input('등록하실분의 이메일을 입력하세요 > ')
    while True:
        try:
            age = int(input('등록하실분의 나이을 정수로 입력하세요 > '))
            grade = int(input('등록하실분의 등급을 정수로 입력하세요 > '))
            break
        except:
            print('정수로 다시 입력해주세요')
    etc = input('등록하실분의 기타사항을 입력하세요 > ')
    cursor.execute('INSERT INTO MEMBER VALUES (:name, :phone, :email, :age, :grade, :etc)', Member(name, phone, email, age, grade, etc).as_dict())
    conn.commit()
    print('입력 성공!')
    cursor.close()

In [6]:
def select_all():
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    for member in members:
        print(to_member(*member))
    cursor.close()

In [7]:
def select_name():
    cursor = conn.cursor()
    name = input('검색하실분의 이름을 작성해 주세요')
    cursor.execute('SELECT * FROM MEMBER WHERE NAME = UPPER(:name)',{'name':name})
    members = cursor.fetchall()
    for member in members:
        print(to_member(*member))
    cursor.close()

In [8]:
def delete_email():
    cursor = conn.cursor()
    email = input('삭제하실분의 이메일을 작성해 주세요')
    cursor.execute('DELETE FROM MEMBER WHERE EMAIL = :email',{'email':email})
    conn.commit()
    if cursor.rowcount >= 1:
        print('삭제완료되었습니다')
    cursor.close()

In [9]:
def save_csv():
    import csv
    cursor = conn.cursor()
    save_member_list = []
    cursor.execute('SELECT * FROM MEMBER')
    members = cursor.fetchall()
    for member in members:
        save_member_list.append(to_member(*member).as_dict())
    fieldnames = list(save_member_list[0].keys())
    filename = input('저장할 CSV 파일 이름은 ?')
    try :
        with open('data/{}.csv'.format(filename), 'w', newline='', encoding='UTF-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
            dict_writer.writeheader()
            dict_writer.writerows(save_member_list)
    except Exception as e:
        print('데이터 내보내기 예외 :', e)
    cursor.close()

In [10]:
def close_sql():
    conn.close()
    print('프로그램이 종료되었습니다')

In [11]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break

In [12]:
if __name__=='__main__':
    global conn
    oracle_dsn = cx_Oracle.makedsn(host='localhost', port=1521, sid='xe')
    conn = cx_Oracle.connect('dlow123', 'tmvjswl', dsn=oracle_dsn)
    main()

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 1
등록하실분의 이름을 입력하세요 > 김길동
등록하실분의 전화번호을 입력하세요 > 010-8888-8888
등록하실분의 이메일을 입력하세요 > k@k.com
등록하실분의 나이을 정수로 입력하세요 > 25
등록하실분의 등급을 정수로 입력하세요 > 2
등록하실분의 기타사항을 입력하세요 > 멍초이
입력 성공!
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 8
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 2
name:홍길동
phone:010-9999-9999
email:h@h.com
age:20
grade:3
etc:바보

name:김길동
phone:010-8888-8888
email:k@k.com
age:25
grade:2
etc:멍초이

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 3
검색하실분의 이름을 작성해 주세요김길동
name:김길동
phone:010-8888-8888
email:k@k.com
age:25
grade:2
etc:멍초이

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 4
삭제하실분의 이메일을 작성해 주세요k@k.com
삭제완료되었습니다
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 0
프로그램이 종료되었습니다


# 연습문제 서술형

```
1. 파이썬의데이터베이스 연동에대해잘 못 설명한 것은?
답 : (4) commit()을 호출하지 않고 데이터베이스 연결 객체를 close() 하면 변경사항은 자동으로 저장된다.

2. 파이썬의Cursor 객체에대해잘 못 설명한 것은?
답 : (2) 커서객체의execute() 메소드는결과set으로 반환한다
```